In [1]:
#standard imports and setups used during EAE127
import math
import numpy as np 
import os
import matplotlib.pyplot as plt 
import matplotlib.lines as mlines
#Disable Python Warning Output
#(NOTE: Only for production, comment out for debugging)
import warnings
warnings.filterwarnings('ignore')
### PLOTTING DEFAULTS BOILERPLATE (OPTIONAL) #########################
#SET DEFAULT FIGURE APPERANCE
import seaborn as sns #Fancy plotting package 
#No Background fill, legend font scale, frame on legend
sns.set_theme(style='whitegrid', font_scale=1.5, rc={'legend.frameon': True})
#Mark ticks with border on all four sides (overrides 'whitegrid')
sns.set_style('ticks')
#ticks point in
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
#fix invisible marker bug
sns.set_context(rc={'lines.markeredgewidth': 0.1})
#restore default matplotlib colormap
mplcolors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
'#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
sns.set_palette(mplcolors)

#Get color cycle for manual colors
colors = sns.color_palette()
#SET MATPLOTLIB DEFAULTS
#(call after seaborn, which changes some defaults)
params = {
#FONT SIZES
'axes.labelsize' : 30, #Axis Labels
'axes.titlesize' : 30, #Title
'font.size' : 28, #Textbox
'xtick.labelsize': 22, #Axis tick labels
'ytick.labelsize': 22, #Axis tick labels
'legend.fontsize': 15, #Legend font size
'font.family' : 'serif',
'font.fantasy' : 'xkcd',
'font.sans-serif': 'Helvetica',
'font.monospace' : 'Courier',
#AXIS PROPERTIES
'axes.titlepad' : 2*6.0, #title spacing from axis
'axes.grid' : True, #grid on plot
'figure.figsize' : (8,8), #square plots
'savefig.bbox' : 'tight', #reduce whitespace in saved figures
#LEGEND PROPERTIES
'legend.framealpha' : 0.5,
'legend.fancybox' : True,
'legend.frameon' : True,
'legend.numpoints' : 1,
'legend.scatterpoints' : 1,
'legend.borderpad' : 0.1,
'legend.borderaxespad' : 0.1,
'legend.handletextpad' : 0.2,
'legend.handlelength' : 1.0,
'legend.labelspacing' : 0,
}
import matplotlib #type:ignore
matplotlib.rcParams.update(params) #update matplotlib defaults, call after￿
### END OF BOILERPLATE ##################################################
colors = sns.color_palette() #color cycle

In [5]:
#constants

#efficiency
etaC = 0.88
etaB = 0.95
etaT = 0.86
etaF = 0.91
etaM = 0.993
etaN = 0.94
etaFN = 0.94
etaAb = 0.9

#total pressure ratio
piC = 17
piF = 2.015127
piD = 0.93
piB = 0.91
piM = 0.98
piAb = 0.96

#ideal heat ratio
kA = 1.408327
kD = 1.40424
kC = 1.386146
kF = 1.400929
kT = 1.337778
kN = 1.280625
kFN = 1.397652

#specific heats (BTU/lbm R)
CpF = 0.2395595
CpC = 0.2461057
CpAb = 0.2810447
CpT = 0.2715298
CpN = 0.3128676
CpFN = 0.2409686
CpB = 0.2677774
CpMC = 0.2569374
CpMu = 0.2466467

deltaH = 18100 #BTU/lbm
Tt4 = 2230 #R
mdot = 157 #lbm/s
Tt6 = 3300 #R
sigma = 0.5
alpha = 1.6
Ma = 0.93

Ta = 420.77 #R
Pa = 4.893503 #psia

R = 53.35 #gas constant



In [51]:
#main code

def calcThrust(etaC=etaC,etaF=etaF,etaT=etaT,etaB=etaB,piB=piB,piD=piD,etaN=etaN):
    #ambient
    aa = np.sqrt(kA*R*32.17*Ta)
    ua = Ma*aa
    Tta = Ta * (1+(kA-1)/2*Ma**2)
    Pta = Pa * (1+(kA-1)/2*Ma**2)**(kA/(kA-1))

    #duct
    Pt2 = Pta*piD
    Tt2 = Tta

    #compressor
    Pt3 = Pt2*piC
    Tt3prime = Tt2 * piC**((kC-1)/kC)
    Tt3 = (Tt3prime - Tt2)/etaC + Tt2

    #combustor
    Pt4 = Pt3*piB
    mdot_f = (mdot*CpB*(Tt4-Tt3))/(deltaH*etaB - CpB*Tt4)

    f = mdot_f/mdot

    #fan
    Pt7 = Pt2*piF
    Tt7prime = Tt2 * piF**((kF-1)/kF)
    Tt7 = (Tt7prime - Tt2)/etaF + Tt2

    #turbine
    Tt5 = Tt4 - (mdot*CpC*(Tt3-Tt2) + alpha*mdot*CpF*(Tt7-Tt2))/(etaM*CpT*(mdot + mdot_f)) #something seems slightly different than hand calcs
    Pt5 = Pt4 * (1 - ((1 - (Tt5/Tt4))/etaT))**(kT/(kT-1))

    #mixer
    Tt7_5 = Tt7
    Pt7_5 = Pt5
    Tt5_5 = ((1+f) * CpMC*Tt5 + sigma*alpha*CpMu*Tt7_5) / ((1+f)*CpMC + sigma*alpha*CpMu)
    Pt5_5 = Pt5 * piM

    #afterburner
    Pt6 = Pt5_5*piAb
    mdot_fab = ((mdot + mdot_f + alpha*sigma*mdot)*CpAb*(Tt6 - Tt5_5)) / (deltaH * etaAb - CpAb*Tt6)
    fab = mdot_fab/mdot

    #primary nozzle
    P8 = Pa
    T8prime = Tt6 * (P8/Pt6) ** ((kN - 1)/kN)
    T8 = Tt6 - etaN * (Tt6 - T8prime)
    a8 = (kN*R*32.17*T8)**(1/2)
    M8 = np.sqrt((1/(etaN * (P8/Pt6) ** ((kN - 1)/kN) + 1 - etaN)-1)*(2/(kN-1)))
    u8 = a8*M8

    #fan nozzle
    P9 = Pa
    T9prime = Tt7 * (P9/Pt7) ** ((kFN - 1)/kFN)
    T9 = Tt7 - etaFN*(Tt7 - T9prime)
    a9 = (kFN*R*32.17*T9) ** (1/2)
    M9 = np.sqrt((1/(etaFN * (P9/Pt7) ** ((kFN - 1)/kFN) + 1 - etaFN)-1)*(2/(kFN-1)))
    u9 = a9*M9

    #thrust
    thrust = (mdot*(1 + f + alpha*sigma + fab)*u8 - mdot*(1 + alpha*sigma)*ua + mdot*alpha*(1-sigma)*(u9 - ua))/32.17
    TSFC = (mdot_f + mdot_fab)/thrust * 3600
    
    print("Thrust: {:.7g} lbf".format(thrust))
    print("TSFC: {:.7g} lbm/lbf hr".format(TSFC))
    print("")

    return thrust, TSFC

calcThrust()
calcThrust(etaC = 0.97)
calcThrust(etaF = 0.97)
calcThrust(etaT = 0.97)
calcThrust(etaB = 0.97)
calcThrust(piB = 0.97)
calcThrust(piD = 0.97)
calcThrust(etaN = 0.97)


Thrust: 23554.94 lbf
TSFC: 2.16976 lbm/lbf hr

Thrust: 25716.84 lbf
TSFC: 1.986594 lbm/lbf hr

Thrust: 23942.79 lbf
TSFC: 2.132465 lbm/lbf hr

Thrust: 26758.77 lbf
TSFC: 1.909975 lbm/lbf hr

Thrust: 23537.62 lbf
TSFC: 2.162434 lbm/lbf hr

Thrust: 24297.88 lbf
TSFC: 2.103418 lbm/lbf hr

Thrust: 24129.3 lbf
TSFC: 2.118113 lbm/lbf hr

Thrust: 24028.76 lbf
TSFC: 2.126975 lbm/lbf hr



(np.float64(24028.764238235657), np.float64(2.1269749539545275))